# **dlt sources and resources**: Create your first dlt pipeline  [![Open in molab](https://marimo.io/molab-shield.svg)](https://molab.marimo.io/github/dlt-hub/dlt/blob/master/docs/education/dlt-fundamentals-course/lesson_2_dlt_sources_and_resources_create_first_dlt_pipeline.py) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dlt-hub/dlt/blob/master/docs/education/dlt-fundamentals-course/lesson_2_dlt_sources_and_resources_create_first_dlt_pipeline.ipynb) [![GitHub badge](https://img.shields.io/badge/github-view_source-2b3137?logo=github)](https://github.com/dlt-hub/dlt/blob/master/docs/education/dlt-fundamentals-course/lesson_2_dlt_sources_and_resources_create_first_dlt_pipeline.ipynb)


![Lesson_2_dlt_sources_and_resources_Create_first_dlt_pipeline_img1.png](https://storage.googleapis.com/dlt-blog-images/dlt-fundamentals-course/Lesson_2_dlt_sources_and_resources_Create_first_dlt_pipeline_img1.png)

## Recap of [Lesson 1](https://github.com/dlt-hub/dlt/blob/master/docs/education/dlt-fundamentals-course/lesson_1_quick_start.ipynb) 👩‍💻🚀
1. Created a pipeline, loaded toy data into DuckDB, and viewed the load info.
2. Used the `dlt.pipeline` and `pipeline.run` methods.
3. Queried data and viewed tables with DuckDB, the `sql_client`, and the dlt `dataset`.

Now, let's move on to the next lesson to learn more! 🚀

**Here, you will learn how to:**
- Run a simple pipeline with different types of data, such as dataframes, databases and REST APIs.
- Use `dlt.resource`, `dlt.source` and `dlt.transformer`.
- Build your first dlt pipeline for a REST API.

In [1]:
%%capture
!pip install dlt

---
## **`dlt` resources**

---
### List of dicts


In the previous lesson, we simply used a list of dictionaries that essentially represents the `pokemon` table.



In [2]:
import dlt

# Sample data containing pokemon details
data = [
    {"id": "1", "name": "bulbasaur", "size": {"weight": 6.9, "height": 0.7}},
    {"id": "4", "name": "charmander", "size": {"weight": 8.5, "height": 0.6}},
    {"id": "25", "name": "pikachu", "size": {"weight": 6, "height": 0.4}},
]


# Set pipeline name, destination, and dataset name
pipeline = dlt.pipeline(
    pipeline_name="quick_start",
    destination="duckdb",
    dataset_name="mydata",
)

# Run the pipeline with data and table name
load_info = pipeline.run(data, table_name="pokemon")

print(load_info)

Pipeline quick_start load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////content/quick_start.duckdb location to store data
Load package 1769045096.6711895 is LOADED and contains no failed jobs


A better way is to wrap it in the `@dlt.resource` decorator which denotes a logical grouping of data within a data source, typically holding data of similar structure and origin:

In [3]:
import dlt
from dlt.common.typing import TDataItems, TDataItem


# Create a dlt resource from the data
@dlt.resource(table_name="pokemon_new")
def my_dict_list() -> TDataItems:
    yield data

Commonly used arguments:

* **`name`**: The resource name and the name of the table generated by this resource. Defaults to the decorated function name.
* **`table_name`**: The name of the table, if different from the resource name.
* **`write_disposition`**: Controls how to write data to a table. Defaults to the value "append".

> **Why is it a better way?** This allows you to use `dlt` functionalities to the fullest that follow Data Engineering best practices, including incremental loading and data contracts.

Try running the pipeline with the `my_dict_list` resource:

In [4]:
# Run the pipeline and print load info
load_info = pipeline.run(my_dict_list)
print(load_info)

Pipeline quick_start load step completed in 0.12 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////content/quick_start.duckdb location to store data
Load package 1769045211.770762 is LOADED and contains no failed jobs


Check what was loaded to the `pokemon_new` table:

In [5]:
pipeline.dataset().pokemon_new.df()

,id,name,size__weight,size__height,_dlt_load_id,_dlt_id
0,1,bulbasaur,6.9,0.7,1769045211.770762,7AvdwUHCAGYEow
1,4,charmander,8.5,0.6,1769045211.770762,+68nHyo2AItsow
2,25,pikachu,6.0,0.4,1769045211.770762,KZO4ErQ4uhiqug


Instead of a dict list, the data could also be a/an:
- dataframe
- database query response
- API request response
- Anything you can transform into JSON/dict format

---
### Dataframes
To create a pipeline using dataframes, you would do:

In [6]:
import pandas as pd


# Define a resource to load data from a CSV
@dlt.resource(table_name="df_data")
def my_df() -> TDataItems:
    sample_df = pd.read_csv("https://people.sc.fsu.edu/~jburkardt/data/csv/hw_200.csv")
    yield sample_df


# Run the pipeline with the defined resource
load_info = pipeline.run(my_df)
print(load_info)

# Query the loaded data from 'df_data'
pipeline.dataset().df_data.df()

Pipeline quick_start load step completed in 0.12 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////content/quick_start.duckdb location to store data
Load package 1769045329.7487633 is LOADED and contains no failed jobs


,index,height_inchesx,_weight_poundsx
0,1,65.78,112.99
1,2,71.52,136.49
2,3,69.40,153.03
3,4,68.22,142.34
4,5,67.79,144.30
...,...,...,...
195,196,65.80,120.84
196,197,66.11,115.78
197,198,68.24,128.30
198,199,68.02,127.47


---
### Databases

To create a pipeline from an SQL database query you would:

In [8]:
%%capture
!pip install pymysql

1. Create and run a pipeline to fetch data from an SQL resource and query the loaded data as follows:

In [9]:
import dlt
from sqlalchemy import create_engine


# Define a resource to fetch genome data from the database
@dlt.resource(table_name="genome_data")
def get_genome_data() -> TDataItems:
    engine = create_engine(
        "mysql+pymysql://rfamro@mysql-rfam-public.ebi.ac.uk:4497/Rfam"
    )
    with engine.connect() as conn:
        query = "SELECT * FROM genome LIMIT 1000"
        rows = conn.execution_options(yield_per=100).exec_driver_sql(query)
        yield from map(lambda row: dict(row._mapping), rows)


# Run the pipeline with the genome data resource
load_info = pipeline.run(get_genome_data)
print(load_info)

# Query the loaded data from 'genome_data'
pipeline.dataset().genome_data.df()

Pipeline quick_start load step completed in 0.98 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////content/quick_start.duckdb location to store data
Load package 1769045713.1600494 is LOADED and contains no failed jobs


,upid,description,total_length,ncbi_id,scientific_name,kingdom,num_rfam_regions,num_families,is_reference,is_representative,...,assembly_acc,assembly_version,assembly_level,ungapped_length,assembly_name,study_ref,circular,wgs_acc,wgs_version,common_name
0,RG000000001,Potato spindle tuber viroid,4591,12892,Potato spindle tuber viroid,viroids,0,0,1,0,...,None,<NA>,None,<NA>,None,None,<NA>,None,<NA>,None
1,RG000000002,Columnea latent viroid,370,12901,Columnea latent viroid,viroids,0,0,1,0,...,None,<NA>,None,<NA>,None,None,<NA>,None,<NA>,None
2,RG000000003,Tomato apical stunt viroid-S,360,53194,Tomato apical stunt viroid-S,viroids,0,0,1,0,...,None,<NA>,None,<NA>,None,None,<NA>,None,<NA>,None
3,RG000000004,Tomato apical stunt viroid,360,12885,Tomato apical stunt viroid,viroids,0,0,1,0,...,None,<NA>,None,<NA>,None,None,<NA>,None,<NA>,None
4,RG000000005,Cucumber yellows virus,7899,32618,Cucumber yellows virus,viruses,0,0,1,0,...,None,<NA>,None,<NA>,None,None,<NA>,None,<NA>,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,UP000001264,None,40275,673838,Enterococcus phage phiFL3B,viruses,0,0,0,1,...,GCA_002630665.1,1,complete-genome,<NA>,ASM263066v1,None,<NA>,None,<NA>,None
996,UP000001265,Enterococcus phage phiFL4A,37856,673839,Enterococcus phage phiFL4A,viruses,2,2,0,1,...,,0,,0,,,0,,0,
997,UP000001266,Staphylococcus phage SA1,147303,694060,Staphylococcus phage SA1,viruses,0,0,0,1,...,,0,,0,,,0,,0,
998,UP000001270,Mycobacterium phage AnnaL29,53253,1076630,Mycobacterium phage AnnaL29,viruses,0,0,0,1,...,,0,,0,,,0,,0,


---
### REST API

For REST API endpoints, create a pipeline as follows:

In [10]:
from dlt.sources.helpers import requests


# Define a resource to fetch pokemons from PokeAPI
@dlt.resource(table_name="pokemon_api")
def get_pokemon() -> TDataItems:
    url = "https://pokeapi.co/api/v2/pokemon"
    response = requests.get(url)
    yield response.json()["results"]


# Run the pipeline using the defined resource
load_info = pipeline.run(get_pokemon)
print(load_info)

# Query the loaded data from 'pokemon_api' table
pipeline.dataset().pokemon_api.df()

Pipeline quick_start load step completed in 0.16 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////content/quick_start.duckdb location to store data
Load package 1769045970.278525 is LOADED and contains no failed jobs


,name,url,_dlt_load_id,_dlt_id
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,1769045970.278525,EHE5SUq+oZdZrA
1,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,1769045970.278525,/pYhy5RG5PGTzw
2,venusaur,https://pokeapi.co/api/v2/pokemon/3/,1769045970.278525,MooOsfCAQH0sog
3,charmander,https://pokeapi.co/api/v2/pokemon/4/,1769045970.278525,pCGL4xJv3bHMeQ
4,charmeleon,https://pokeapi.co/api/v2/pokemon/5/,1769045970.278525,+DoPMk+WaDSIIQ
5,charizard,https://pokeapi.co/api/v2/pokemon/6/,1769045970.278525,thDLAzFIS9vuHw
6,squirtle,https://pokeapi.co/api/v2/pokemon/7/,1769045970.278525,1+zTPRtZGgdyHg
7,wartortle,https://pokeapi.co/api/v2/pokemon/8/,1769045970.278525,aLs/ZPqoBIjMlQ
8,blastoise,https://pokeapi.co/api/v2/pokemon/9/,1769045970.278525,0QxHPDlHtP37eA
9,caterpie,https://pokeapi.co/api/v2/pokemon/10/,1769045970.278525,V+Ukoz8QwH4EMw


Try loading everything above, in a single pipeline:

In [14]:
load_info = pipeline.run([my_df, get_genome_data, get_pokemon])
print(load_info)

Pipeline quick_start load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////content/quick_start.duckdb location to store data
Load package 1769046083.4786613 is LOADED and contains no failed jobs


Check which new tables were created:

In [15]:
# List all table names from the database
with pipeline.sql_client() as client:
    with client.execute_query(
        "SELECT table_name FROM information_schema.tables"
    ) as table:
        print(table.df())

            table_name
0              df_data
1          genome_data
2              pokemon
3          pokemon_api
4          pokemon_new
5           _dlt_loads
6  _dlt_pipeline_state
7         _dlt_version



---
## **`dlt` sources**

Now that there are multiple `dlt` resources, each corresponding to a separate table, we can group them into a `dlt` source.


![Lesson_2_dlt_sources_and_resources_Create_first_dlt_pipeline_img2](https://storage.googleapis.com/dlt-blog-images/dlt-fundamentals-course/Lesson_2_dlt_sources_and_resources_Create_first_dlt_pipeline_img2.png)

A source is a logical grouping of resources, e.g., endpoints of a single API. The most common approach is to define it in a separate Python module.

* A source is a function decorated with `@dlt.source` that returns one or more resources.
* A source can optionally define a schema with tables, columns, performance hints, and more.
* The source Python module typically contains optional customizations and data transformations.
* The source Python module typically contains the authentication and pagination code for a particular API.

Read more about [sources](https://dlthub.com/docs/general-usage/source) and [resources](https://dlthub.com/docs/general-usage/resource).

You declare a source by decorating a function that returns or yields one or more resources with `@dlt.source`.

Here's how it's done:

In [16]:
from typing import Iterable
from dlt.extract import DltResource


@dlt.source
def all_data() -> Iterable[DltResource]:
    return my_df, get_genome_data, get_pokemon

Only using the source above, load everything into a separate database using a new pipeline:

In [17]:
# Create a pipeline
new_pipeline = dlt.pipeline(
    pipeline_name="resource_source_new", destination="duckdb", dataset_name="all_data"
)

# Run the pipeline
load_info = new_pipeline.run(all_data())

# Print load info
print(load_info)

Pipeline resource_source_new load step completed in 0.32 seconds
1 load package(s) were loaded to destination duckdb and into dataset all_data
The duckdb destination used duckdb:////content/resource_source_new.duckdb location to store data
Load package 1769046427.2930565 is LOADED and contains no failed jobs


> **Why does this matter?**:
- It is more efficient than running your resources separately.
- It organizes both your schema and your code. 🙂
- It enables the option for parallelization.

---
## **`dlt` transformers**

We now know that `dlt` resources can be grouped into a `dlt` source, represented as:


```
                  Source
               /          \
          Resource 1  ...  Resource N

```

However, imagine a scenario where you need an additional step in between:

```
                  Source
                 /     \
             step        \
             /             \
        Resource 1  ...  Resource N

```

This step could arise, for example, in a situation where:

- Resource 1 returns a list of pokemons IDs, and you need to use each of those IDs to retrieve detailed information about the pokemons from a separate API endpoint.

In such cases, you would use `dlt` transformers — special `dlt` resources that can be fed data from another resource:

```
                  Source
                 /     \
          Transformer    \
             /             \
        Resource 1  ...  Resource N

```


Let’s assume Resource 1 is:

In [18]:
@dlt.resource(table_name="pokemon")
def my_pokemons() -> TDataItems:
    pokemons = [
        {"id": "1", "name": "bulbasaur", "size": {"weight": 6.9, "height": 0.7}},
        {"id": "4", "name": "charmander", "size": {"weight": 8.5, "height": 0.6}},
        {"id": "25", "name": "pikachu", "size": {"weight": 6, "height": 0.4}},
    ]
    yield pokemons

We need to get detailed information about pokemons from [PokeAPI](https://pokeapi.co/) `"https://pokeapi.co/api/v2/pokemon/{id}"` based on their IDs. We would do:

In [19]:
# Define a transformer to enrich pokemon data with additional details
# NOTE: the `items` argument contains data from the `my_dict_list` resource
@dlt.transformer(data_from=my_pokemons, table_name="detailed_info")
def poke_details(
    items: TDataItems,
) -> TDataItems:
    for item in items:
        print(f"Item: {item}\n")

        item_id = item["id"]
        url = f"https://pokeapi.co/api/v2/pokemon/{item_id}"
        response = requests.get(url)
        details = response.json()

        print(f"Details: {details}\n")

        yield details


# Set pipeline name, destination, and dataset name
another_pipeline = dlt.pipeline(
    pipeline_name="quick_start",
    destination="duckdb",
    dataset_name="pokedata",
    dev_mode=True,
)

Run the pipeline:

In [20]:
load_info = another_pipeline.run(poke_details())
print(load_info)

Item: {'id': '1', 'name': 'bulbasaur', 'size': {'weight': 6.9, 'height': 0.7}}

Details: {'abilities': [{'ability': {'name': 'overgrow', 'url': 'https://pokeapi.co/api/v2/ability/65/'}, 'is_hidden': False, 'slot': 1}, {'ability': {'name': 'chlorophyll', 'url': 'https://pokeapi.co/api/v2/ability/34/'}, 'is_hidden': True, 'slot': 3}], 'base_experience': 64, 'cries': {'latest': 'https://raw.githubusercontent.com/PokeAPI/cries/main/cries/pokemon/latest/1.ogg', 'legacy': 'https://raw.githubusercontent.com/PokeAPI/cries/main/cries/pokemon/legacy/1.ogg'}, 'forms': [{'name': 'bulbasaur', 'url': 'https://pokeapi.co/api/v2/pokemon-form/1/'}], 'game_indices': [{'game_index': 153, 'version': {'name': 'red', 'url': 'https://pokeapi.co/api/v2/version/1/'}}, {'game_index': 153, 'version': {'name': 'blue', 'url': 'https://pokeapi.co/api/v2/version/2/'}}, {'game_index': 153, 'version': {'name': 'yellow', 'url': 'https://pokeapi.co/api/v2/version/3/'}}, {'game_index': 1, 'version': {'name': 'gold', 'url

2026-01-22 01:56:00,819|[WARNING]|716|134579716989568|dlt|validate.py|verify_normalized_table:91|In schema `quick_start`: The following columns in table 'detailed_info' did not receive any data during this load and therefore could not have their types inferred:
  - sprites__other__dream_world__front_female
  - sprites__other__showdown__back_shiny_female
  - sprites__versions__generation_ix__scarlet_violet__front_female
  - sprites__versions__generation_vii__icons__front_female
  - sprites__versions__generation_viii__brilliant_diamond_shining_pearl__front_female

Unless type hints are provided, these columns will not be materialized in the destination.
One way to provide type hints is to use the 'columns' argument in the '@dlt.resource' decorator.  For example:

@dlt.resource(columns={'sprites__other__dream_world__front_female': {'data_type': 'text'}})

2026-01-22 01:56:00,822|[WARNING]|716|134579716989568|dlt|validate.py|verify_normalized_table:91|In schema `quick_start`: The following

Pipeline quick_start load step completed in 0.70 seconds
1 load package(s) were loaded to destination duckdb and into dataset pokedata_20260122015355
The duckdb destination used duckdb:////content/quick_start.duckdb location to store data
Load package 1769046960.4597867 is LOADED and contains no failed jobs


Alternatively, we could do:

In [21]:
@dlt.resource(table_name="pokemon")
def my_other_pokemons() -> TDataItems:
    pokemons = [
        {"id": "1", "name": "bulbasaur", "size": {"weight": 6.9, "height": 0.7}},
        {"id": "4", "name": "charmander", "size": {"weight": 8.5, "height": 0.6}},
        {"id": "25", "name": "pikachu", "size": {"weight": 6, "height": 0.4}},
    ]
    yield from pokemons


# NOTE: Transformer receives one item at a time
@dlt.transformer(data_from=my_other_pokemons, table_name="detailed_info")
def other_poke_details(
    data_item: TDataItem,
) -> TDataItems:
    item_id = data_item["id"]
    url = f"https://pokeapi.co/api/v2/pokemon/{item_id}"
    response = requests.get(url)
    details = response.json()

    yield details


load_info = another_pipeline.run(other_poke_details())
print(load_info)

2026-01-22 01:57:03,231|[WARNING]|716|134579716989568|dlt|validate.py|verify_normalized_table:91|In schema `quick_start`: The following columns in table 'detailed_info' did not receive any data during this load and therefore could not have their types inferred:
  - sprites__other__dream_world__front_female
  - sprites__other__showdown__back_shiny_female
  - sprites__versions__generation_ix__scarlet_violet__front_female
  - sprites__versions__generation_vii__icons__front_female
  - sprites__versions__generation_viii__brilliant_diamond_shining_pearl__front_female

Unless type hints are provided, these columns will not be materialized in the destination.
One way to provide type hints is to use the 'columns' argument in the '@dlt.resource' decorator.  For example:

@dlt.resource(columns={'sprites__other__dream_world__front_female': {'data_type': 'text'}})

2026-01-22 01:57:03,232|[WARNING]|716|134579716989568|dlt|validate.py|verify_normalized_table:91|In schema `quick_start`: The following

Pipeline quick_start load step completed in 0.74 seconds
1 load package(s) were loaded to destination duckdb and into dataset pokedata_20260122015355
The duckdb destination used duckdb:////content/quick_start.duckdb location to store data
Load package 1769047022.963123 is LOADED and contains no failed jobs


You can also use pipe instead of `data_from`, this is useful when you want to apply `dlt.transformer` to multiple `dlt.resources`:

In [22]:
load_info = another_pipeline.run(my_pokemons | poke_details)
print(load_info)

Item: {'id': '1', 'name': 'bulbasaur', 'size': {'weight': 6.9, 'height': 0.7}}

Details: {'abilities': [{'ability': {'name': 'overgrow', 'url': 'https://pokeapi.co/api/v2/ability/65/'}, 'is_hidden': False, 'slot': 1}, {'ability': {'name': 'chlorophyll', 'url': 'https://pokeapi.co/api/v2/ability/34/'}, 'is_hidden': True, 'slot': 3}], 'base_experience': 64, 'cries': {'latest': 'https://raw.githubusercontent.com/PokeAPI/cries/main/cries/pokemon/latest/1.ogg', 'legacy': 'https://raw.githubusercontent.com/PokeAPI/cries/main/cries/pokemon/legacy/1.ogg'}, 'forms': [{'name': 'bulbasaur', 'url': 'https://pokeapi.co/api/v2/pokemon-form/1/'}], 'game_indices': [{'game_index': 153, 'version': {'name': 'red', 'url': 'https://pokeapi.co/api/v2/version/1/'}}, {'game_index': 153, 'version': {'name': 'blue', 'url': 'https://pokeapi.co/api/v2/version/2/'}}, {'game_index': 153, 'version': {'name': 'yellow', 'url': 'https://pokeapi.co/api/v2/version/3/'}}, {'game_index': 1, 'version': {'name': 'gold', 'url

2026-01-22 01:57:23,334|[WARNING]|716|134579716989568|dlt|validate.py|verify_normalized_table:91|In schema `quick_start`: The following columns in table 'detailed_info' did not receive any data during this load and therefore could not have their types inferred:
  - sprites__other__dream_world__front_female
  - sprites__other__showdown__back_shiny_female
  - sprites__versions__generation_ix__scarlet_violet__front_female
  - sprites__versions__generation_vii__icons__front_female
  - sprites__versions__generation_viii__brilliant_diamond_shining_pearl__front_female

Unless type hints are provided, these columns will not be materialized in the destination.
One way to provide type hints is to use the 'columns' argument in the '@dlt.resource' decorator.  For example:

@dlt.resource(columns={'sprites__other__dream_world__front_female': {'data_type': 'text'}})

2026-01-22 01:57:23,334|[WARNING]|716|134579716989568|dlt|validate.py|verify_normalized_table:91|In schema `quick_start`: The following

Pipeline quick_start load step completed in 0.71 seconds
1 load package(s) were loaded to destination duckdb and into dataset pokedata_20260122015355
The duckdb destination used duckdb:////content/quick_start.duckdb location to store data
Load package 1769047042.8695223 is LOADED and contains no failed jobs


Check the loaded data:

In [23]:
# Query the 'detailed_info' table and convert the result to a DataFrame
another_pipeline.dataset().detailed_info.df()

,base_experience,cries__latest,cries__legacy,height,id,is_default,location_area_encounters,name,order,species__name,...,sprites__versions__generation_v__black_white__back_shiny_female,sprites__versions__generation_v__black_white__front_female,sprites__versions__generation_v__black_white__front_shiny_female,sprites__versions__generation_vi__omegaruby_alphasapphire__front_female,sprites__versions__generation_vi__omegaruby_alphasapphire__front_shiny_female,sprites__versions__generation_vi__x_y__front_female,sprites__versions__generation_vi__x_y__front_shiny_female,sprites__versions__generation_vii__ultra_sun_ultra_moon__front_female,sprites__versions__generation_vii__ultra_sun_ultra_moon__front_shiny_female,sprites__versions__generation_viii__icons__front_female
0,64,https://raw.githubusercontent.com/PokeAPI/crie...,https://raw.githubusercontent.com/PokeAPI/crie...,7,1,True,https://pokeapi.co/api/v2/pokemon/1/encounters,bulbasaur,1,bulbasaur,...,None,None,None,None,None,None,None,None,None,None
1,62,https://raw.githubusercontent.com/PokeAPI/crie...,https://raw.githubusercontent.com/PokeAPI/crie...,6,4,True,https://pokeapi.co/api/v2/pokemon/4/encounters,charmander,5,charmander,...,None,None,None,None,None,None,None,None,None,None
2,112,https://raw.githubusercontent.com/PokeAPI/crie...,https://raw.githubusercontent.com/PokeAPI/crie...,4,25,True,https://pokeapi.co/api/v2/pokemon/25/encounters,pikachu,35,pikachu,...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
3,64,https://raw.githubusercontent.com/PokeAPI/crie...,https://raw.githubusercontent.com/PokeAPI/crie...,7,1,True,https://pokeapi.co/api/v2/pokemon/1/encounters,bulbasaur,1,bulbasaur,...,None,None,None,None,None,None,None,None,None,None
4,62,https://raw.githubusercontent.com/PokeAPI/crie...,https://raw.githubusercontent.com/PokeAPI/crie...,6,4,True,https://pokeapi.co/api/v2/pokemon/4/encounters,charmander,5,charmander,...,None,None,None,None,None,None,None,None,None,None
5,112,https://raw.githubusercontent.com/PokeAPI/crie...,https://raw.githubusercontent.com/PokeAPI/crie...,4,25,True,https://pokeapi.co/api/v2/pokemon/25/encounters,pikachu,35,pikachu,...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
6,64,https://raw.githubusercontent.com/PokeAPI/crie...,https://raw.githubusercontent.com/PokeAPI/crie...,7,1,True,https://pokeapi.co/api/v2/pokemon/1/encounters,bulbasaur,1,bulbasaur,...,None,None,None,None,None,None,None,None,None,None
7,62,https://raw.githubusercontent.com/PokeAPI/crie...,https://raw.githubusercontent.com/PokeAPI/crie...,6,4,True,https://pokeapi.co/api/v2/pokemon/4/encounters,charmander,5,charmander,...,None,None,None,None,None,None,None,None,None,None
8,112,https://raw.githubusercontent.com/PokeAPI/crie...,https://raw.githubusercontent.com/PokeAPI/crie...,4,25,True,https://pokeapi.co/api/v2/pokemon/25/encounters,pikachu,35,pikachu,...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI

---
## **Reduce the nesting level of generated tables**
You can limit how deep dlt goes when generating nested tables and flattening dicts into columns. By default, the library will descend and generate nested tables for all nested lists, without limit.

You can set nesting level for all resources on the source level:

```python
@dlt.source(max_table_nesting=1)
def all_data():
  return my_df, get_genome_data, get_pokemon
```

or for each resource separately:

```python
@dlt.resource(table_name='pokemon_new', max_table_nesting=1)
def my_dict_list():
    yield data
```

In the example above, we want only 1 level of nested tables to be generated (so there are no nested tables of a nested table). Typical settings:

* `max_table_nesting=0` will not generate nested tables and will not flatten dicts into columns at all. All nested data will be represented as JSON.
* `max_table_nesting=1` will generate nested tables of root tables and nothing more. All nested data in nested tables will be represented as JSON.

---
## **Exercise 1: Create a pipeline for GitHub API – repos endpoint**

In this exercise, you'll build a dlt pipeline to fetch data from the GitHub REST API. The goal is to learn how to use `dlt.pipeline`, `dlt.resource`, and `dlt.source` to extract and load data into a destination.

## Instructions

1. **Explore the GitHub API**

   Visit the [GitHub REST API Docs](https://docs.github.com/en/rest) to understand the endpoint to [list public repositories](https://docs.github.com/en/rest/repos/repos?apiVersion=2022-11-28) for an organization:

   `GET https://api.github.com/orgs/{org}/repos`

2. **Build the pipeline**

   Write a script to:

   - Fetch repositories for the **dlt-hub** organization.
   - Use `dlt.resource` to define the data extraction logic.
   - Combine all resources into a single `@dlt.source`.
   - Load the data into a DuckDB database.

3. **Inspect the data**

   Use a `duckdb` connection, `sql_client`, or `pipeline.dataset()`.

> **Note**: For this exercise you don't need to use authentication or pagination.


Play with the API using the requests library:


In [24]:
from dlt.sources.helpers import requests

response = requests.get("https://api.github.com/orgs/dlt-hub/repos")
response.json()[0]

{'id': 427664318,
 'node_id': 'R_kgDOGX2jvg',
 'name': 'rasa_semantic_schema',
 'full_name': 'dlt-hub/rasa_semantic_schema',
 'private': False,
 'owner': {'login': 'dlt-hub',
  'id': 89419010,
  'node_id': 'MDEyOk9yZ2FuaXphdGlvbjg5NDE5MDEw',
  'avatar_url': 'https://avatars.githubusercontent.com/u/89419010?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/dlt-hub',
  'html_url': 'https://github.com/dlt-hub',
  'followers_url': 'https://api.github.com/users/dlt-hub/followers',
  'following_url': 'https://api.github.com/users/dlt-hub/following{/other_user}',
  'gists_url': 'https://api.github.com/users/dlt-hub/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/dlt-hub/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/dlt-hub/subscriptions',
  'organizations_url': 'https://api.github.com/users/dlt-hub/orgs',
  'repos_url': 'https://api.github.com/users/dlt-hub/repos',
  'events_url': 'https://api.github.com/users/dlt-hub/events{/pr

In the code snippet below, you will find an **example** for the **`events`** endpoint:

In [25]:
import dlt


# Example resource
@dlt.resource
def github_events() -> TDataItems:
    url = "https://api.github.com/orgs/dlt-hub/events"
    response = requests.get(url)
    yield response.json()


print("build the `github_repos` resource here")


@dlt.source
def github_data() -> Iterable[DltResource]:
    return (github_events,)


print("return your new resource as part of the source above")


# Set pipeline name, destination, and dataset name
github_pipeline = dlt.pipeline(
    pipeline_name="github_pipeline", destination="duckdb", dataset_name="github_data"
)

load_info = github_pipeline.run(github_data())
print(load_info)

build the `github_repos` resource here
return your new resource as part of the source above


2026-01-22 02:09:00,745|[WARNING]|716|134579716989568|dlt|validate.py|verify_normalized_table:91|In schema `github_data`: The following columns in table 'github_events' did not receive any data during this load and therefore could not have their types inferred:
  - payload__issue__milestone
  - payload__issue__type
  - payload__issue__active_lock_reason
  - payload__issue__performed_via_github_app
  - payload__comment__performed_via_github_app
  - payload__forkee__language
  - payload__forkee__mirror_url
  - payload__forkee__license

Unless type hints are provided, these columns will not be materialized in the destination.
One way to provide type hints is to use the 'columns' argument in the '@dlt.resource' decorator.  For example:

@dlt.resource(columns={'payload__issue__milestone': {'data_type': 'text'}})



Pipeline github_pipeline load step completed in 0.48 seconds
1 load package(s) were loaded to destination duckdb and into dataset github_data
The duckdb destination used duckdb:////content/github_pipeline.duckdb location to store data
Load package 1769047740.0495224 is LOADED and contains no failed jobs


In [27]:
import dlt

@dlt.resource(table_name="github_repos")
def github_repos() -> TDataItems:
  url = "https://api.github.com/orgs/dlt-hub/repos"
  response = requests.get(url)
  yield response.json()

@dlt.source
def github_data():
  return github_events, github_repos

load_info = github_pipeline.run(github_data())
print(load_info)

2026-01-22 02:13:04,545|[WARNING]|716|134579716989568|dlt|validate.py|verify_normalized_table:91|In schema `github_data`: The following columns in table 'github_repos' did not receive any data during this load and therefore could not have their types inferred:
  - mirror_url

Unless type hints are provided, these columns will not be materialized in the destination.
One way to provide type hints is to use the 'columns' argument in the '@dlt.resource' decorator.  For example:

@dlt.resource(columns={'mirror_url': {'data_type': 'text'}})

2026-01-22 02:13:04,546|[WARNING]|716|134579716989568|dlt|validate.py|verify_normalized_table:91|In schema `github_data`: The following columns in table 'github_events' did not receive any data during this load and therefore could not have their types inferred:
  - payload__issue__milestone
  - payload__issue__type
  - payload__issue__active_lock_reason
  - payload__issue__performed_via_github_app
  - payload__forkee__language
  - payload__forkee__mirror

Pipeline github_pipeline load step completed in 0.66 seconds
1 load package(s) were loaded to destination duckdb and into dataset github_data
The duckdb destination used duckdb:////content/github_pipeline.duckdb location to store data
Load package 1769047983.072527 is LOADED and contains no failed jobs


In [33]:
with github_pipeline.sql_client() as client:
  with client.execute_query("SELECT COUNT(*) FROM github_repos") as table:
    print(table.df())

   count_star()
0            30


### Question
How many columns has the `github_repos` table? Use a `duckdb` connection, `sql_client` or `pipeline.dataset()`.

## **Exercise 2: Create a pipeline for the GitHub API – stargazers endpoint**

Create a `dlt.transformer` for the **"stargazers"** endpoint  
`https://api.github.com/repos/OWNER/REPO/stargazers` for the `dlt-hub` organization.

Use the `github_repos` resource as the main resource for the transformer:

1. Get all repositories in the `dlt-hub` organization.  
2. Feed these repository names into the `dlt` transformer and retrieve all stargazers for all `dlt-hub` repositories.


In [41]:
print("YOUR CODE GOES HERE")

@dlt.transformer(data_from=github_repos, table_name="github_stargazer")
def github_stargazers(
    items: TDataItems,
) -> TDataItems:
    for item in items:
      item_id = item["id"]
      item_name = item["name"]
      url = f"https://api.github.com/repos/dlt-hub/{item_name}/stargazers"
      response = requests.get(url)
      details = response.json()
      yield details

YOUR CODE GOES HERE


In [42]:
github_pipeline.run(github_stargazers())

LoadInfo(pipeline=<dlt.pipeline(pipeline_name='github_pipeline', destination='duckdb', dataset_name='github_data', default_schema_name='github_data', schema_names=['github_data'], first_run=False, dev_mode=False, is_active=True, pipelines_dir='/var/dlt/pipelines', working_dir='/var/dlt/pipelines/github_pipeline')>, metrics={'1769049008.3258488': [{'started_at': DateTime(2026, 1, 22, 2, 30, 14, 382449, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2026, 1, 22, 2, 30, 14, 699425, tzinfo=Timezone('UTC')), 'job_metrics': {'github_stargazer.964745cbed.insert_values.gz': LoadJobMetrics(job_id='github_stargazer.964745cbed.insert_values.gz', file_path='/var/dlt/pipelines/github_pipeline/load/normalized/1769049008.3258488/started_jobs/github_stargazer.964745cbed.0.insert_values.gz', table_name='github_stargazer', started_at=DateTime(2026, 1, 22, 2, 30, 14, 480792, tzinfo=Timezone('UTC')), finished_at=DateTime(2026, 1, 22, 2, 30, 14, 532049, tzinfo=Timezone('UTC')), state='completed', remote_

In [44]:
with github_pipeline.sql_client() as client:
  with client.execute_query("SELECT COUNT(*) FROM github_stargazer") as table:
    print(table.df())

   count_star()
0           168


### Question
How many columns has the `github_stargazer` table? Use a `duckdb` connection, `sql_client` or `pipeline.dataset()`.

✅ ▶ Proceed to the [next lesson](https://github.com/dlt-hub/dlt/blob/master/docs/education/dlt-fundamentals-course/lesson_3_pagination_and_authentication_and_dlt_configuration.ipynb)!